# Triples2Text

### Imports

Used the Benchmark object that was supplied with the data to parse the data files.

In [1]:
from utils.benchmark_reader import Benchmark
from utils.benchmark_reader import select_files

b = Benchmark()
files = select_files("./data/mt_train.xml")
b.fill_benchmark(files)
initial_data_dict = b.to_dict()


<li> Familiarised myself with the dict. 
<li> Flattened the first dimension of the dict, as it had only 1 element, so that dimension was redundant.

In [2]:
initial_data_dict = initial_data_dict['entries']

<li> Found that the dict had length 13211.

In [3]:
len(initial_data_dict)

13211

The dict was in the form of a list, where each element in the list held a dict with 1 element. This could also be flattened. The dicts started from index 1, so that was fixed and reverted to 0.

In [4]:
flattened_dict = {}

for dictionary in initial_data_dict:
    key, value = list(dictionary.items())[0]

    flattened_dict[key-1] = value

From reading the documentation, I understood that I should be using the 'modifiedtripleset' element as a triple, as this is the most readable and accurate. <br> <br>
'lexicalisations' is a list of dictionaries with correct results. These results are in both English and Maltese depending on the 'lang' attribute.

# x

get data

In [5]:
import spacy
import string

change sm to lg?

In [6]:
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 9999999

Tokenization, Lemmatization and Excluding Punctuation

In [7]:
first10 = {}
for x in range(10):
    first10.update({x: flattened_dict[x]})

getting and tokenizing modifiedtripleset & lexicalisations

In [26]:
def tokenize_lemmatize_punct(triple_value):
    return [token.lemma_ for token in nlp(triple_value) if token.text not in string.punctuation]

In [50]:
def token_and_punct_of_whole_dict(flattened_dict):
    result = []
    for idx in range(len(flattened_dict)):
        # initialising lists
        all_triples_per_dict_entry = []
        processed_item = {}
        lexes = []

        #iterating through list of triples and tokenising
        for triple_set in flattened_dict[idx]['modifiedtripleset']:
            processed_item = {
            'subject': tokenize_lemmatize_punct(triple_set['subject']),
            'property': tokenize_lemmatize_punct(triple_set['property']),
            'object': tokenize_lemmatize_punct(triple_set['object'])
            }
            all_triples_per_dict_entry.append(processed_item)

        #iterating through list of lexicalisations
        for lexicalisation in flattened_dict[idx]['lexicalisations']:
            #if language is
            if lexicalisation['lang'] == '':
                #the cleaned lexicalisation is appended to a list
                lexes.append(tokenize_lemmatize_punct(lexicalisation ['lex']))

        result.append([all_triples_per_dict_entry, lexes])

    return result

In [51]:
token_punct = token_and_punct_of_whole_dict(flattened_dict)

In [8]:
import json

In [78]:
json_object = json.dumps(token_punct)

# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [10]:
with open('sample.json', 'r') as openfile:

    # Reading from json file
    token_punct = json.load(openfile)

extracting data

In [11]:
#returns list of list of dicts
def get_triples_from_dict(flattened_dict):
    result = []
    for idx in range(len(flattened_dict)):
        result.append(flattened_dict[idx]['modifiedtripleset'])
    return result

In [12]:
triple_lists = get_triples_from_dict(flattened_dict)

In [14]:
def get_vocab(cleaned_data):
    vocab = set()

    for entry in cleaned_data:
        triple = entry[0]
        lex = entry[1]

        for current_triple in triple:
            for token in current_triple['object']:
                vocab.add(token)
            for token in current_triple['property']:
                vocab.add(token)
            for token in current_triple['subject']:
                vocab.add(token)

        #add every token in lexicalisation
        for current_lex in lex:
            for token in current_lex:
                vocab.add(token)
    return vocab

In [15]:
vocab_list = get_vocab(token_punct)

One-Hot

In [18]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [13]:
encoder = OneHotEncoder()

In [29]:
data = {'word':list(vocab_list)}

In [30]:
df = pd.DataFrame(data)

In [31]:
one_hot_array = encoder.fit_transform(df[['word']]).toarray()

In [36]:
one_hot_df = pd.DataFrame(one_hot_array, columns=encoder.get_feature_names_out())

Next?

In [42]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [43]:
def model_creation(vocabulary_size, sequence_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, sequence_len, input_length=sequence_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    model.summary()

    return model

In [ ]:
model_creation(vocab_list)